# ML infill demonstrations

Automunge is available now for pip install:

In [1]:
# !pip install Automunge

Or to upgrade (we currently roll out upgrades pretty frequently):

In [2]:
# !pip install Automunge --upgrade

Once installed, run this in a local session to initialize:

In [1]:
from Automunge import *
am = AutoMunge()

ML infill is a push button method for missing data inputation, utilizing auto ML models trained on partitions of the training set for each feature. These models trained on the designated training set and saved in the returned postprocess_dict dictionary can then be applied to infer infill to training and test data passed to the automunge(.) function as well as to subsequent test data passed to the postmunge(.) function.

ML infill need not only be applied in scenarios with missing data present. It can also be applied to fully represented data sets to serve as a precaution against imperfections in subsequent data streams.

ML infill is turned on by default with the automunge(.) parameter setting `MLinfill=True`. It can be deactivate by passing `MLinfill=False`, in which case assigning to distinct columns can take place in `assigninfill` parameter.

The default architecture is Random Forest for simplicity, although other auto ML architectures like AutoGluon and Catboost are also available which may make use of ensembles and train on GPU's for more accurate and quickly trained models.

In this notebook we'll demonstrate how simple applying ML infill is in context of an automunge(.) call, as well as demonstrate passing parameters to the model training and selecting alternate architectures.

To demonstrate, let's encode the Ames Housing set, a well known tabular benchmark:

In [2]:
import pandas as pd

#housing set
df_train = pd.read_csv('housing_train.csv')
df_test = pd.read_csv('housing_test.csv')

labels_column = 'SalePrice'
trainID_column = 'Id'

Here is what the data looks like in a raw form.

In [5]:
pd.set_option('display.max_columns', 300)
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Note that in missing data theory, there is a fundamental distinction between values that are missing at random and values that are not missing at random. For example, in this set, the 'Alley' column appears to contain NaN values associated with houses that are not located on an alley, so this might not be as good of a target for ML infill, amd may be better suited for a distinct value as infill. In order to signal to training the presence of cells that were subject to infill, we'll activate the NArw_marker parameter, which will assemble a column for each feature set containing boolean integer markers signalling presence of entries that were subject to infill. (This parameter is activated by default).

This data set does not have a very high prevelance of missing at random data, so to improve visibility we'll apply an automunge(.) parameter that can inject missing data into a training set. Let's arbitrarily select for injection the columns 'LotArea' and 'Neighborhood'.

In [3]:
#this is to inject some missing at random entries into the selected columns
#for demonstration of ML infill
#we'll inject missing data into 50% of entries in each target column

assignnan = {}

assignnan.update({'injections' : {'LotArea'      : {'inject_ratio' : .5},
                                  'Neighborhood' : {'inject_ratio' : .5}}})

Now let's go ahead and run ML infill using just defaults on this set.

In [7]:
train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assignnan = assignnan,
               NArw_marker = True,
               shuffletrain = False, 
               MLinfill = True,
               printstatus = False
              )

The returned data looks like following, note the NArw columns signalling presence of infill.

In [8]:
train.head()

,MSSubClass_nmbr,LotFrontage_nmbr,LotArea_nmbr,Street_bnry,Alley_bnry,Utilities_bnry,OverallQual_nmbr,OverallCond_nmbr,YearBuilt_nmbr,YearRemodAdd_nmbr,MasVnrArea_nmbr,BsmtFinSF1_nmbr,BsmtFinSF2_nmbr,BsmtUnfSF_nmbr,TotalBsmtSF_nmbr,CentralAir_bnry,1stFlrSF_nmbr,2ndFlrSF_nmbr,LowQualFinSF_nmbr,GrLivArea_nmbr,BsmtFullBath_nmbr,FullBath_nmbr,BedroomAbvGr_nmbr,KitchenAbvGr_nmbr,TotRmsAbvGrd_nmbr,Fireplaces_nmbr,GarageYrBlt_nmbr,GarageCars_nmbr,GarageArea_nmbr,WoodDeckSF_nmbr,OpenPorchSF_nmbr,EnclosedPorch_nmbr,3SsnPorch_nmbr,ScreenPorch_nmbr,PoolArea_nmbr,MiscVal_nmbr,MoSold_nmbr,YrSold_nmbr,MSSubClass_NArw,MSZoning_NArw,MSZoning_1010_0,MSZoning_1010_1,MSZoning_1010_2,LotFrontage_NArw,LotArea_NArw,Street_NArw,Alley_NArw,LotShape_NArw,LotShape_1010_0,LotShape_1010_1,LotShape_1010_2,LandContour_NArw,LandContour_1010_0,LandContour_1010_1,LandContour_1010_2,Utilities_NArw,LotConfig_NArw,LotConfig_1010_0,LotConfig_1010_1,LotConfig_1010_2,LandSlope_NArw,LandSlope_1010_0,LandSlope_1010_1,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4,Condition1_NArw,Condition1_1010_0,Condition1_1010_1,Condition1_1010_2,Condition1_1010_3,Condition2_NArw,Condition2_1010_0,Condition2_1010_1,Condition2_1010_2,Condition2_1010_3,BldgType_NArw,BldgType_1010_0,BldgType_1010_1,BldgType_1010_2,HouseStyle_NArw,HouseStyle_1010_0,HouseStyle_1010_1,HouseStyle_1010_2,HouseStyle_1010_3,OverallQual_NArw,OverallCond_NArw,YearBuilt_NArw,YearRemodAdd_NArw,RoofStyle_NArw,RoofStyle_1010_0,RoofStyle_1010_1,RoofStyle_1010_2,RoofMatl_NArw,RoofMatl_1010_0,RoofMatl_1010_1,RoofMatl_1010_2,RoofMatl_1010_3,Exterior1st_NArw,Exterior1st_1010_0,Exterior1st_1010_1,Exterior1st_1010_2,Exterior1st_1010_3,Exterior2nd_NArw,Exterior2nd_1010_0,Exterior2nd_1010_1,Exterior2nd_1010_2,Exterior2nd_1010_3,Exterior2nd_1010_4,MasVnrType_NArw,MasVnrType_1010_0,MasVnrType_1010_1,MasVnrType_1010_2,MasVnrArea_NArw,ExterQual_NArw,ExterQual_1010_0,ExterQual_1010_1,ExterQual_1010_2,ExterCond_NArw,ExterCond_1010_0,ExterCond_1010_1,ExterCond_1010_2,Foundation_NArw,Foundation_1010_0,Foundation_1010_1,Foundation_1010_2,BsmtQual_NArw,BsmtQual_1010_0,BsmtQual_1010_1,BsmtQual_1010_2,BsmtCond_NArw,BsmtCond_1010_0,BsmtCond_1010_1,BsmtCond_1010_2,BsmtExposure_NArw,BsmtExposure_1010_0,BsmtExposure_1010_1,BsmtExposure_1010_2,BsmtFinType1_NArw,BsmtFinType1_1010_0,BsmtFinType1_1010_1,BsmtFinType1_1010_2,BsmtFinSF1_NArw,BsmtFinType2_NArw,BsmtFinType2_1010_0,BsmtFinType2_1010_1,BsmtFinType2_1010_2,BsmtFinSF2_NArw,BsmtUnfSF_NArw,TotalBsmtSF_NArw,Heating_NArw,Heating_1010_0,Heating_1010_1,Heating_1010_2,HeatingQC_NArw,HeatingQC_1010_0,HeatingQC_1010_1,HeatingQC_1010_2,CentralAir_NArw,Electrical_NArw,Electrical_1010_0,Electrical_1010_1,Electrical_1010_2,1stFlrSF_NArw,2ndFlrSF_NArw,LowQualFinSF_NArw,GrLivArea_NArw,BsmtFullBath_NArw,BsmtHalfBath_NArw,BsmtHalfBath_1010_0,BsmtHalfBath_1010_1,FullBath_NArw,HalfBath_NArw,HalfBath_1010_0,HalfBath_1010_1,BedroomAbvGr_NArw,KitchenAbvGr_NArw,KitchenQual_NArw,KitchenQual_1010_0,KitchenQual_1010_1,KitchenQual_1010_2,TotRmsAbvGrd_NArw,Functional_NArw,Functional_1010_0,Functional_1010_1,Functional_1010_2,Fireplaces_NArw,FireplaceQu_NArw,FireplaceQu_1010_0,FireplaceQu_1010_1,FireplaceQu_1010_2,GarageType_NArw,GarageType_1010_0,GarageType_1010_1,GarageType_1010_2,GarageYrBlt_NArw,GarageFinish_NArw,GarageFinish_1010_0,GarageFinish_1010_1,GarageCars_NArw,GarageArea_NArw,GarageQual_NArw,GarageQual_1010_0,GarageQual_1010_1,GarageQual_1010_2,GarageCond_NArw,GarageCond_1010_0,GarageCond_1010_1,GarageCond_1010_2,PavedDrive_NArw,PavedDrive_1010_0,PavedDrive_1010_1,WoodDeckSF_NArw,OpenPorchSF_NArw,EnclosedPorch_NArw,3SsnPorch_NArw,ScreenPorch_NArw,PoolArea_NArw,PoolQC_NArw,PoolQC_1010_0,PoolQC_1010_1,Fence_NArw,Fence_1010_0,Fence_1010_1,Fence_1010_2,MiscFeature_NArw,MiscFeature_1010_0,MiscFeature_1010_1,MiscFeature_1010_2,MiscVal_NArw,MoSold_NArw,YrSold_NArw,SaleType_NArw,SaleType_1010_0,SaleType_1010_1,SaleType_1010_2,SaleType_1010

The returned columns in some cases don't have grouping coherence for those derived from the same source feature. This was an intentional design decision for memory overhead, if grouping coherence is desired for the returned sets can activated by passing a parameter to assignparam as:
```
assignparam = {'global_assignparam' : {'inplace' : False}}
```
Since we are interested in viewing together columns originating from the same source column, we can do so with the column map saved in the returned postprocess_dict.

In [9]:
train[postprocess_dict['column_map']['LotArea'] + \
      postprocess_dict['column_map']['Neighborhood']].head()

,LotArea_nmbr,LotArea_NArw,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4
0,-0.241283,0,1,0,0,0,0,0
1,0.058864,1,0,1,1,0,0,0
2,-0.097692,1,1,0,0,0,0,0
3,-0.093364,0,0,0,0,1,1,0
4,0.191731,1,0,0,1,1,1,1


Here we see inputation in a few rows, where 'LotArea_NArw' signals those entries of 'LotArea' that were subject to infill, and 'Neighborhood_NArw' signals those entries of 'Neighborhood' that were sunject to infill. (Again we injected a few nan points into the set for demonstration purposes.)

The same models from the train set basis are saved and returned in the postprocess_dict, allowing for a consistent inputation basis to subsequent data in the postmunge(.) function.

In [10]:
test, test_ID, test_labels, \
postreports_dict \
= am.postmunge(postprocess_dict, 
               df_test,
               printstatus = False)

In [11]:
test[postprocess_dict['column_map']['LotArea'] + \
      postprocess_dict['column_map']['Neighborhood']].head()

,LotArea_nmbr,LotArea_NArw,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4
0,-0.212333,1,1,0,0,0,0,0
1,0.540939,0,1,0,0,0,0,0
2,0.023684,1,1,0,0,0,0,0
3,0.021091,1,1,0,0,0,0,0
4,-0.704539,0,1,0,0,0,0,0


If we want to apply ML infill only to select columns, we can deactivate the default with `MLinfill=False` and assign to distinct columns in `assigninfill` parameter. Note that column assignments can either be performed with source column headers or returned column headers which include suffix appenders.

Here we'll demonstrate applying ML infill to a subset of the columns.

In [12]:
train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               NArw_marker = True,
               shuffletrain = False, 
               MLinfill = False,
               assigninfill = {'MLinfill':['LotArea', 'Neighborhood']},
               printstatus = False
              )

The Random Forest ML infill can either be applied with defaults, or can also accept parameters passed to the training operation by way of the ML_cmnd parameter. Parameters are consistent with the Scikit Learn Random forest Classifier and Regressor that are the basis. Let's demonstrate here increasing the number of estimators (which will increase training time). Random Forest is pretty good for tendency not to overfit from what I understand.

In [13]:
#here we'll increase n_estimators from default of 100
ML_cmnd = {'autoML_type'  :'randomforest', \
           'MLinfill_cmnd':{'RandomForestClassifier':{'n_estimators' : 1000}, \
                            'RandomForestRegressor' :{'n_estimators' : 1000}}
          }

In [14]:
train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assignnan = assignnan,
               ML_cmnd = ML_cmnd,
               NArw_marker = True,
               shuffletrain = False, 
               MLinfill = True,
               printstatus = False
              )

train[postprocess_dict['column_map']['LotArea'] + \
      postprocess_dict['column_map']['Neighborhood']].head()

,LotArea_nmbr,LotArea_NArw,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4
0,-0.282955,0,0,0,0,1,0,1
1,-0.011420,1,0,1,1,0,0,0
2,-0.228283,1,0,0,0,1,0,1
3,-0.026828,1,1,0,0,0,0,0
4,0.452619,0,1,0,0,0,0,0


Of course there are tradeoffs associated with Random Forest, like the accuracy may not be as high as some other methods like gradient boosting and does not support GPU training which may be needed for larger training sets. Automunge doesn't currently include a gradient boosting option, partly because hyperparameter selection is a little more tricky and higher risk of overfitting, but we do have a few auto ML options that are expected to perform at comparable or better.

Catboost: One way to think about Catboost is that it is an extension of gradient boosting, including support for GPU's, wrapped in an auto ML package.

AutoGluon: Autogluon is another auto ML package that takes a different approach than catboost, relying on aggregation of ensembles (ensembles which include catboost within as a matter of fact).

FLAML: FLAML is an auto ML library with some unique approaches for hyperparameter tuning. This library is expected to perform efficiently in inference. We'll demonstrate below designating a training time limit for each feature by parameter.

We can activate one of these other ML infill architectures in ML_Cmnd as well. (Note that you'll need to install them first, instructions are available on their respective websites).

In [15]:
#And here we demonstrate applying ML infill with CatBoost

ML_cmnd = {'autoML_type':'catboost'}

train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assignnan = assignnan,
               ML_cmnd = ML_cmnd,
               NArw_marker = True,
               shuffletrain = False, 
               MLinfill = True,
               printstatus = False
              )

train[postprocess_dict['column_map']['LotArea'] + \
      postprocess_dict['column_map']['Neighborhood']].head()

,LotArea_nmbr,LotArea_NArw,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4
0,-0.259700,1,1,1,1,0,0,1
1,-0.120471,0,1,1,1,0,0,1
2,0.077376,0,1,1,1,0,0,1
3,-0.126466,0,0,0,0,1,1,0
4,0.118578,1,0,0,1,1,1,1


In [16]:
#Here we demonstrate applying ML infill with AutoGluon

ML_cmnd = {'autoML_type':'autogluon', \
           'MLinfill_cmnd':{'AutoGluon'  : {'verbosity' : 0}}}

train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assignnan = assignnan,
               ML_cmnd = ML_cmnd,
               NArw_marker = True,
               shuffletrain = False, 
               MLinfill = True,
               printstatus = False
              )

train[postprocess_dict['column_map']['LotArea'] + \
      postprocess_dict['column_map']['Neighborhood']].head()

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 865, in _train_and_save
    score = model.score(X=X_val, y=y_val)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 283, in score
    y_pred = self.predict(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 244, in predict
    y_pred_proba = self.predict_proba(X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 251, in predict_proba
    y_pred_proba = self._predict_proba(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mli

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 856, in _train_and_save
    model = self._train_single(X_train, y_train, model, X_val, y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 827, in _train_single
    model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 233, in fit
    self._fit(**kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/lgb/lgb_model.py", line 152, in _fit
    self.model = lgb.train(**train_params)
  File "/Users/nicholasteague/opt/anaconda3/envs/

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 865, in _train_and_save
    score = model.score(X=X_val, y=y_val)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 283, in score
    y_pred = self.predict(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 244, in predict
    y_pred_proba = self.predict_proba(X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 251, in predict_proba
    y_pred_proba = self._predict_proba(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mli

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 856, in _train_and_save
    model = self._train_single(X_train, y_train, model, X_val, y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 827, in _train_single
    model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 233, in fit
    self._fit(**kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/lgb/lgb_model.py", line 152, in _fit
    self.model = lgb.train(**train_params)
  File "/Users/nicholasteague/opt/anaconda3/envs/

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 865, in _train_and_save
    score = model.score(X=X_val, y=y_val)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 283, in score
    y_pred = self.predict(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 244, in predict
    y_pred_proba = self.predict_proba(X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 251, in predict_proba
    y_pred_proba = self._predict_proba(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mli

...

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 865, in _train_and_save
    score = model.score(X=X_val, y=y_val)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 283, in score
    y_pred = self.predict(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 244, in predict
    y_pred_proba = self.predict_proba(X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 251, in predict_proba
    y_pred_proba = self._predict_proba(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mli

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 856, in _train_and_save
    model = self._train_single(X_train, y_train, model, X_val, y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 827, in _train_single
    model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 233, in fit
    self._fit(**kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/lgb/lgb_model.py", line 152, in _fit
    self.model = lgb.train(**train_params)
  File "/Users/nicholasteague/opt/anaconda3/envs/

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 865, in _train_and_save
    score = model.score(X=X_val, y=y_val)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 283, in score
    y_pred = self.predict(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 244, in predict
    y_pred_proba = self.predict_proba(X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 251, in predict_proba
    y_pred_proba = self._predict_proba(X=X, preprocess=preprocess)
  File "/Users/nicholasteague/opt/anaconda3/envs/mli

Traceback (most recent call last):
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 856, in _train_and_save
    model = self._train_single(X_train, y_train, model, X_val, y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/trainer/abstract_trainer.py", line 827, in _train_single
    model.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **model_fit_kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/abstract/abstract_model.py", line 233, in fit
    self._fit(**kwargs)
  File "/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/autogluon/utils/tabular/ml/models/lgb/lgb_model.py", line 152, in _fit
    self.model = lgb.train(**train_params)
  File "/Users/nicholasteague/opt/anaconda3/envs/

,LotArea_nmbr,LotArea_NArw,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4
0,-0.213533,1,1,0,0,0,0,0
1,-0.031697,1,0,1,0,1,1,1
2,-0.108329,1,0,0,0,1,0,0
3,-0.136284,0,0,0,0,1,0,1
4,0.410957,0,0,0,1,1,1,0


In [4]:
#And here we demonstrate applying ML infill with FLAML

ML_cmnd = {'autoML_type':'flaml'}

#we can also pass parameters to designate a limit on training time for each feature (in seconds)
ML_cmnd.update({'MLinfill_cmnd' : {'flaml_classifier_fit'   : {'time_budget' : 10 },
                                   'flaml_regressor_fit'    : {'time_budget' : 10}}})

train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assignnan = assignnan,
               ML_cmnd = ML_cmnd,
               NArw_marker = True,
               shuffletrain = False, 
               MLinfill = True,
               printstatus = False
              )

train[postprocess_dict['column_map']['LotArea'] + \
      postprocess_dict['column_map']['Neighborhood']].head()

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

...

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/opt/anaconda3/envs/mlinfill/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/nicholasteague/op

,LotArea_nmbr,LotArea_NArw,Neighborhood_NArw,Neighborhood_1010_0,Neighborhood_1010_1,Neighborhood_1010_2,Neighborhood_1010_3,Neighborhood_1010_4
0,-0.400192,0,0,0,0,1,0,1
1,-0.172678,0,1,0,1,1,1,0
2,0.147621,1,0,0,0,1,0,1
3,-0.340247,1,0,0,0,1,1,0
4,0.749245,0,0,0,1,1,1,1
